In [ ]:
import math
import numpy as np
import pandas as pd

In [ ]:
# df = pd.read_csv("../data/money/money_df.csv")
df = pd.read_csv("../data/diabetes/diabetes.csv")


In [ ]:
data = []
classes = []
targetColIdx = -1
numAttributes = -1 
maxCategory = 5
attrValues = {}
attributes = [] #done
tree = None

In [ ]:
attributes = list(df.drop(df.columns[targetColIdx], axis = 1).columns)


In [ ]:
attrValues = {}
for attribute in attributes:
    unique_value = df.loc[:, attribute].unique()
    if len(unique_value) > maxCategory: 
        #numeric
        attrValues[attribute] = ['continuous']
    else:
        #categoric
        attrValues[attribute] = sorted(df.loc[:, attribute].astype(str).unique())
        df.loc[:,attribute] = df.loc[:, attribute].astype(str)
df.loc[:,df.columns[targetColIdx]] = df.loc[:, df.columns[targetColIdx]].astype(str)

In [ ]:
data = df.values.tolist()

In [ ]:
classes = sorted(df.iloc[:,targetColIdx].astype(str).unique())

In [ ]:
def isAttrDiscrete(attribute):
    if attribute not in attributes:
        raise ValueError("Attribute not listed")
    elif len(attrValues[attribute]) == 1 and attrValues[attribute][0] == "continuous":
        return False
    else:
        return True

In [ ]:
def log(x):
    if x == 0:
        return 0
    else:
        return math.log(x,2)

In [ ]:
def entropy(dataSet):
    S = len(dataSet)
    if S == 0:
        return 0
    num_classes = [0 for i in classes]
    for row in dataSet:
        classIndex = list(classes).index(row[-1])
        num_classes[classIndex] += 1
    num_classes = [x/S for x in num_classes]
    ent = 0
    for num in num_classes:
        ent += num*log(num)
    return ent*-1

In [ ]:
def gain(unionSet, subsets):
    #input : data and disjoint subsets of it
    #output : information gain
    S = len(unionSet)
    #calculate impurity before split
    impurityBeforeSplit = entropy(unionSet)
    
    #calculate impurity after split
    weights = [len(subset)/S for subset in subsets]
    impurityAfterSplit = 0
    splitInfo = 0
#     for i in range(len(subsets)):
        
#         splitInfo -= weights[i] * log(weights[i])
#         print("weights[{i}] * log(weights[{i}]) : ({a}/{b})  * log({a}/{b})".format(i = i, 
#                                                     a = len(subsets[i]),
#                                                     b = S))
#         impurityAfterSplit += weights[i]*entropy(subsets[i])
#         print("weights[{i}]*entropy(subsets[{i}]): ({a}/{b}) * {entropy}".format(i = i, 
#                                                     weight = weights[i], 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     entropy = entropy(subsets[i])))
#     print("\timpurityAfterSplit")
    for i in range(len(subsets)):
        impurityAfterSplit += weights[i]*entropy(subsets[i])
#         print("weights[{i}]*entropy(subsets[{i}]): ({a}/{b}) * {entropy} = {result}".format(i = i, 
#                                                     weight = weights[i], 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     entropy = entropy(subsets[i]),
#                                                     result = weights[i]*entropy(subsets[i])))
#     print("\tsplitInfo")
    for i in range(len(subsets)):

        splitInfo -= weights[i] * log(weights[i]) 
#         print("weights[{i}] * log(weights[{i}]) : ({a}/{b})  * log({a}/{b}) = {result}".format(i = i, 
#                                                     a = len(subsets[i]),
#                                                     b = S,
#                                                     result = weights[i] * log(weights[i])))
        
    #calculate total gain
    totalGain = (impurityBeforeSplit - impurityAfterSplit)
    if splitInfo == 0:
        gainRatio = 0 
    else:
        gainRatio = (impurityBeforeSplit - impurityAfterSplit) / splitInfo
    
#     print("gainRatio = (impurityBeforeSplit - impurityAfterSplit) / splitInfo : \n {gainRatio} = ({impurityBeforeSplit} - {impurityAfterSplit}) / {splitInfo}".format(gainRatio = gainRatio, impurityBeforeSplit = impurityBeforeSplit, impurityAfterSplit = impurityAfterSplit, splitInfo = splitInfo))
#     print("totalGain : {}".format(totalGain))
    
#     return gainRatio
    return totalGain


In [ ]:
def allSameClass(data):
    if len(data) == 0:
        return False
    for row in data:
        if row[-1] != data[0][-1]:
            return False
    return data[0][-1]

In [ ]:
def getClassComposition(curData):
    class_composition = [0]*len(classes)
    if len(curData) > 0:
        for row in curData:
            index = classes.index(row[-1])
            class_composition[index] += 1
            
    return class_composition

In [ ]:
def getMajClass(class_composition):
    maxInd = class_composition.index(max(class_composition))
    return classes[maxInd]


In [ ]:
class Node:
    def __init__(self, isLeaf, parent_label, label, split_on, threshold, train_composition, valid_composition):
        self.isLeaf = isLeaf
        self.parent_label = parent_label
        self.label = label
        self.split_on = split_on
        self.threshold = threshold
        self.children = []
        self.train_composition = train_composition 
        self.valid_composition = valid_composition 
        

In [ ]:
def splitAttribute(curData, curAttributes):
    splitted = []
    maxEnt = -1*float("inf")
    best_attribute = -1
    #None for discrete attributes, threshold value for continuous attributes
    best_threshold = None
    for attribute in curAttributes:
        indexOfAttribute = attributes.index(attribute)
#         print("\nAttribute : {}".format(attribute))
        if isAttrDiscrete(attribute):
            #split curData into n-subsets, where n is the number of 
            #different values of attribute i. Choose the attribute with
            #the max gain
            valuesForAttribute = attrValues[attribute]
            subsets = [[] for a in valuesForAttribute]
            for row in curData:
                for index in range(len(valuesForAttribute)):
                    if row[indexOfAttribute] == valuesForAttribute[index]:
                        subsets[index].append(row)
                        break
            e = gain(curData, subsets)
            if e > maxEnt:
                maxEnt = e
                splitted = subsets
                best_attribute = attribute
                best_threshold = None
        else:
            #sort the data according to the column.Then try all 
            #possible adjacent pairs. Choose the one that 
            #yields maximum gain
            curData.sort(key = lambda x: x[indexOfAttribute])
            for j in range(0, len(curData) - 1):
                if curData[j][indexOfAttribute] != curData[j+1][indexOfAttribute]:
                    threshold = (curData[j][indexOfAttribute] + curData[j+1][indexOfAttribute]) / 2
                    less = []
                    greater = []
                    for row in curData:
                        if(row[indexOfAttribute] > threshold):
                            greater.append(row)
                        else:
                            less.append(row)
#                     print("threshold: {}".format(threshold))
                    e = gain(curData, [less, greater])
                    if e >= maxEnt:
                        splitted = [less, greater]
                        maxEnt = e
                        best_attribute = attribute
                        best_threshold = threshold
    return (best_attribute,best_threshold,splitted)


In [ ]:
def recursiveGenerateTree(curData, curAttributes, parent_label):
    allSame = allSameClass(curData)
    class_composition = getClassComposition(curData)
    majClass = getMajClass(class_composition)
    
    if len(curData) == 0:
        #Empty data 
        return Node(True, parent_label, parent_label, None, None, class_composition, None)
    
    else: 

        
        if allSame is not False:
            #return a node with that class
            return Node(True, parent_label, allSame, None, None, class_composition, None)
        elif len(curAttributes) == 0:
            #return a node with the majority class
            return Node(True, parent_label, majClass, None, None,class_composition, None)
        else:
            #Not a leaf
            (best_attribute,best_threshold,splitted) = splitAttribute(curData, curAttributes)
            remainingAttributes = curAttributes[:]
            if best_attribute in remainingAttributes:
                remainingAttributes.remove(best_attribute)

            node = Node(False, parent_label, majClass, best_attribute, best_threshold, class_composition, None)
            node.children = [recursiveGenerateTree(subset, remainingAttributes, majClass) for subset in splitted
                            ]
            if len(node.children) == 0:
                node.isLeaf = True
            return node
        
        

In [ ]:
def printNode(node, indent=""):
    if not node.isLeaf:
        if node.threshold is None:
            #discrete
            for index,child in enumerate(node.children):
                if child.isLeaf:
#                     if child.label != 'Empty':
                        print(indent + node.split_on + " = " + attrValues[node.split_on][index] + " : " + child.label)
                else:
                    print(indent + node.split_on + " = " + attrValues[node.split_on][index] + " : ")
                    printNode(child, indent + "	")
        else:
            #numerical
            leftChild = node.children[0]
            rightChild = node.children[1]
            if leftChild.isLeaf:
#                 if leftChild.label != 'Empty':
                    print(indent + node.split_on + " <= " + str(node.threshold) + " : " + leftChild.label)
            else:
                print(indent + node.split_on + " <= " + str(node.threshold)+" : ")
                printNode(leftChild, indent + "	")

            if rightChild.isLeaf:
#                 if rightChild.label != 'Empty':
                    print(indent + node.split_on + " > " + str(node.threshold) + " : " + rightChild.label)
            else:
                print(indent + node.split_on + " > " + str(node.threshold) + " : ")
                printNode(rightChild , indent + "	")


In [ ]:
def printNodeTrainComposition(node, indent=""):
    
    if not node.isLeaf:
        if node.threshold is None:
            #discrete
            for index,child in enumerate(node.children):
                if child.isLeaf:
#                     if child.label != 'Empty':
                        print(indent + " " + node.split_on + " = " + attrValues[node.split_on][index] + " : " + child.label + " " + str(child.train_composition))
                else:
                    print(indent + " " + node.split_on + " = " + attrValues[node.split_on][index] + " : " + str(child.train_composition))
                    printNodeTrainComposition(child, indent = indent + "	")
        else:
            #numerical
            leftChild = node.children[0]
            rightChild = node.children[1]
            if leftChild.isLeaf:
#                 if leftChild.label != 'Empty':
                    print(indent + " " + node.split_on + " <= " + str(node.threshold) + " : " + leftChild.label + " " + str(leftChild.train_composition))
            else:
                print(indent + " " + node.split_on + " <= " + str(node.threshold)+" : " + str(leftChild.train_composition))
                printNodeTrainComposition(leftChild, indent = indent + "	")

            if rightChild.isLeaf:
#                 if rightChild.label != 'Empty':
                    print(indent + " " + node.split_on + " > " + str(node.threshold) + " : " + rightChild.label + " " +str(rightChild.train_composition))
            else:
                print(indent + " " + node.split_on + " > " + str(node.threshold) + " : " + str(rightChild.train_composition))
                printNodeTrainComposition(rightChild , indent = indent + "	")


In [ ]:
def predictOne(input, node):
    if node.isLeaf: 
#         print(node.label)
        return node.label
    else: 
        split_on = node.split_on
        input_value = input[attributes.index(split_on)]
        if node.threshold is None:
            #discrete
            child_index = attrValues[split_on].index(input_value)
            child_node = node.children[child_index]
#             print(split_on, child_index)
            return predictOne(input, child_node)
        else: 
            #numerical
            threshold_value = node.threshold
            if input_value <= threshold_value:
                child_node = node.children[0]
            else:
                child_node = node.children[1]
#             print(split_on, input_value, threshold_value)
            return predictOne(input, child_node)
            
def predict(inputs, node):
    return list(map(predictOne, inputs, [node] * len(inputs)))
        

In [ ]:
def evaluate(data, node):
    prediction = np.array(predict(data, node))
    real_target = np.array(data)[:,-1]
    accuracy = sum(real_target == prediction) / len(data)
    return accuracy
    

In [ ]:
tree = recursiveGenerateTree(curData = data, curAttributes = attributes, parent_label = None)

In [ ]:
# def printTree(self):
printNodeTrainComposition(tree)

In [ ]:
evaluate(data, tree) # akurasi data training

In [ ]:
# def printTree(self):
printNodeTrainComposition(tree)